This is where i'm doing my research, I've included it into the github because I see no reason not to but if you're looking for a place to begin I would recommend cretin_main

In [6]:
from importlib import reload
import sys
import numpy as np
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file #these python classes should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, plt_file]:
    reload(obj)

In [7]:
search_for_word = 'source'
search_inside_chapter = 'Radiation'
#df = search.Generators().word(input = search_for_word, word_slicing = (0,10), print_path = True)
#print(df.to_string())
#print(search.Generators().chapter(input = search_inside_chapter))


In [8]:
# first we create our generic object
def sim(var1, var2, var3):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9, 30, 512  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 5 # the laser energies
    DN, DR = 10, .1


    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-7, restart = True)

    gen2.source_laser(laser_wavelength = var2, option_1= 'rate', option_2= 'history', values = [1,var2], nodes= [N0, N1])
    gen2.source_jnu(E_range=[E1,var3], option_1= 'rate', option_2= 'history', nodes= [N0, var1], values = [1,var2])
    gen2.source_jbndry(index = 1, E_range=[E1,var2], option_1= 'rate', option_2= 'history', nodes = [1,var2], values=[1,var2])


    gen2.popular_switches(continuum_transfer= 'do steady-state and use integral formalism formalism', timestep_between_snapshot = 3)
    return gen2


i0, iend = 0,10
trials = []
name = 'trial'
for i in range(i0,iend):

    trial = name+str(i)
    trials.append(trial)
    var1 = 3
    var2 = 4**i
    var3 = 1.3
    gen2 = sim(var1, var2, var3)
    #write_run_plot.write(name = name+str(i), object = gen2)
    #write_run_plot.run(name = name+str(i), longprint=False)

#serial_sim_tools.plot_all('trials_mutiplot', trials)

In [14]:
def tin(source, variable):

    gen2 = generator_object.User_input()

    N0, N1 = 1, 121 #number of nodes is 20
    DR =  .1
    TE, rho = 30.0, .002
    E1, E2 = 4.68, 4.68*1.001
    HPEV = 4.135667516e-15
    mult =  HPEV / (E2 - E1)
    RSPOT = .01
    ELASER, TLASER = .06, 2e-9
    ILASER = variable*10**7*ELASER/(TLASER*np.pi*RSPOT**2)
    
    gen2.materials_atom(index = 1, quantum_n_max = 5, element= "sn")    

    gen2.materials_region(nodes = [N0, N1], elec_temp = TE)
    gen2.materials_region_rho(rho)
    gen2.materials_region_element(index = 1, initial_ion_population = 1.)


    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 101, start = 0.1, end = 10**(2))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end = 3e-9, restart = True)
    gen2.controls_history(id = 1, value_mutiplier = ILASER, time_multiplier = TLASER)
    gen2.controls_history_tv(time = 0., value = 1)
    gen2.controls_history_tv(time = 1.0, value = 1.)
    gen2.controls_history_tv(time = 1.0, value = 0.)


    gen2.parameters(scattering_muliplier = 0,initial_timestep = 10**-13, minimum_timestep = 10**-14, maximum_timestep = 10**-9, time_between_snapshots=10**(-9))
    gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], max_iterations_per_timestep=20)

    gen2.add_plot(name = 'plotter', xvar = 'time', yvar = 'tiv')
    gen2.add_plot(name = 'plotter1', xvar = 'time', yvar = 'tev')
    gen2.add_plot(name = 'plotter2', xvar = 'time', yvar = 'rho')
    gen2.add_plot(name = 'plotter3', xvar = 'time', yvar = 'tev', element_or_transition='1:N:DN')
    gen2.add_plot(name = 'plotter5', xvar = 'time', yvar = 'zbar')
    gen2.add_plot(name = 'plotter6', xvar = 'time', yvar = 'nion')
    gen2.add_plot(name = 'plotter7', xvar = 'time', yvar = 'ne')

    if source == 'SOURCE1':
        gen2.source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', values = [1,1], nodes= [N0, N1])

    elif source == 'SOURCE2':
        gen2.source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', values = [1,mult], nodes= [N0, N1])

    elif source == 'SOURCE3':
        gen2.source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.source_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = mult)
        gen2.popular_switches(temparture_calc_heating_rates=['temp calc = time dependant','heating rates = electronic'], 
                              max_iterations_per_timestep = 10, continuum_transfer='do steady-state and use integral formalism formalism')
        
    elif source == 'SOURCE4':
        gen2.source_jbndry(index = 1, E_range= [E1, E2], option_1= 'rate', option_2= 'history',values = [1,mult])
        gen2.source_boundary(package='radiation', type = 'streaming', nodes= [1, 0, N1, 'xfile'], value = 1, mult = mult)
        gen2.popular_switches(continuum_transfer_evolves_temp = True)
        gen2.source_rswitch(radiation_transfer_algorithm1d= 'do transport using integral formalism', assume_NLTE= True)  
    
    return gen2


i0 = 18
trials = []
name = 'trial'
N, M = 6, 2 # this is the multiplot group size and number of groups
iend = i0+M*N

for i in range(i0,iend):
    name_i = name+str(i)
    trials.append(name_i)
    variable = 2**(i/2)
    gen2 = tin('SOURCE1', variable)

    write_run_plot.write(name = name_i, object = gen2)
    write_run_plot.run(name = name_i, longprint=False)

    if (i>0 and i%N == 0):
        serial_sim_tools.plot_all(f'trials_mutiplot{int(i/N)}', trials)
        trials = []

    elif i == iend -1:
        serial_sim_tools.plot_all(f'trials_mutiplot{int(i/N)+1}', trials)

including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial18/trial18.gen
running cretin with trial18
multiplot to /home/brewster/Desktop/bowie/trials_mutiplot3
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial19/trial19.gen
running cretin with trial19
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add plots"

writing to /home/brewster/Desktop/bowie/trial20/trial20.gen
running cretin with trial20
including some of "element_or_transition, node, frequency_or_isosequence, direction_or_level, multiplier" is ambigious and may lead to incorrect behaviour in "add p